# Import CoFID Dataset

In [8]:
import pandas as pd

# import CoFID dataset

cofid_path = 'data/CoFID_Dataset_2021.xlsx'

# proximates sheet (traditional food composition data)
df_proximates = pd.read_excel(cofid_path, sheet_name='1.3 Proximates', usecols='B, H:AU')
use_cols_prox = ['Food Name', 'Water (g)', 'Protein (g)', 'Fat (g)', 'Carbohydrate (g)', 
            'Energy (kcal) (kcal)', 'Starch (g)', 'Total sugars (g)', 'AOAC fibre (g)']
df_proximates = df_proximates[use_cols_prox]
df_proximates = df_proximates.rename(columns={'Energy (kcal) (kcal)': 'Energy (kcal)', 
                                             'AOAC fibre (g)': 'Fibre (g)',
                                             'Total sugars (g)': 'Sugars (g)', 
                                             'Food Name': 'food'})

# inorganics sheet (metals and minerals like potassium, calcium, etc.)
df_inorganics = pd.read_excel(cofid_path, sheet_name='1.4 Inorganics', usecols='B, H:S')
df_inorganics = df_inorganics.rename(columns={'Food Name': 'food'})

# Merge tables
df_cofid = pd.merge(df_proximates, df_inorganics, on="food", how="outer").fillna(0)

print(df_cofid.columns)

#display(df_proximates.head())
#display(df_inorganics.head())


Index(['food', 'Water (g)', 'Protein (g)', 'Fat (g)', 'Carbohydrate (g)',
       'Energy (kcal)', 'Starch (g)', 'Sugars (g)', 'Fibre (g)', 'Sodium (mg)',
       'Potassium (mg)', 'Calcium (mg)', 'Magnesium (mg)', 'Phosphorus (mg)',
       'Iron (mg)', 'Copper (mg)', 'Zinc (mg)', 'Chloride (mg)',
       'Manganese (mg)', 'Selenium (µg)', 'Iodine (µg)'],
      dtype='object')


In [9]:
# search cofid dataset through this link 
#https://quadram.ac.uk/UKfoodcomposition/?s=Chopped+tomatoes&submit=Search

food_grams = {
    'Bananas, flesh only' : 114, 
    'Raspberries, raw' : 100, 
    'Passion fruit, flesh and pips' : 22, 
    'Pumpkin seeds' : 10,
    'Yogurt, low fat, plain' : 70,
    'Chicken, breast, grilled without skin, meat only' : 360, 
    'Ham, gammon joint, boiled' : 57, 
    'Tomatoes, canned, whole contents' : 400, 
    'Baked beans, canned in tomato sauce, reduced sugar, reduced salt' : 400, 
    'Avocado, Hass, flesh only' : 45, 
    'Almonds, whole kernels' : 12, 
    'Tuna, canned in brine, drained' : 50, 
    'Chocolate, plain' : 40
}

In [14]:
num_cols = df_cofid.columns.drop('food')
df_cofid[num_cols] = df_cofid[num_cols]\
    .apply(pd.to_numeric, errors='coerce')

# 2️⃣ Then your loop will work without TypeErrors:
records = []
for food, grams in food_grams.items():
    match = df_cofid[df_cofid['food'].str.contains(food, na=False)]
    if match.empty:
        rec = {'food': food, 'grams': grams}
        for col in num_cols:
            rec[col] = None
    else:
        row = match.iloc[0]
        rec = {'food': food, 'grams': grams}
        for col in num_cols:
            # row[col] is now a float or NaN, so this math works
            rec[col] = row[col] * grams / 100
    records.append(rec)

df_items = pd.DataFrame(records).set_index('food').round(1)
totals   = df_items.sum(numeric_only=True).round(1)

#print("Per-item breakdown:")
#print(df_items, "\n")
print("Daily totals:")
print(totals)

Daily totals:
grams               1680.0
Water (g)           1258.9
Protein (g)          179.2
Fat (g)               48.6
Carbohydrate (g)     132.5
Energy (kcal)       1648.7
Starch (g)            45.4
Sugars (g)            86.4
Fibre (g)             31.0
Sodium (mg)         1852.9
Potassium (mg)      5145.7
Calcium (mg)         440.3
Magnesium (mg)       485.0
Phosphorus (mg)     2111.1
Iron (mg)             14.3
Copper (mg)            2.3
Zinc (mg)             10.2
Chloride (mg)       3960.2
Manganese (mg)         3.2
Selenium (µg)        115.0
Iodine (µg)           81.3
dtype: float64


In [15]:
rda = {
    "Energy (kcal)":      2500,   # average adult male energy needs
    "Protein (g)":        56,     # 0.8 g/kg for ~70 kg male
    "Carbohydrate (g)":   130,    # minimum carbohydrate RDA
    "Fibre (g)":          38,     # Adequate Intake for men 19-50 y
    "Sodium (mg)":        2300,   # Tolerable Upper Intake Level
    "Potassium (mg)":     3400,   # Adequate Intake for men
    "Calcium (mg)":       1000,   # RDA for adults 19-50 y
    "Magnesium (mg)":     420,    # DV for adults
    "Phosphorus (mg)":    700,    # RDA for adults 19+ y
    "Iron (mg)":          8,      # RDA for men
    "Copper (mg)":        0.9,    # RDA for adults
    "Zinc (mg)":          11      # RDA for adult males
}

# 2️⃣ Build DataFrame for comparison
# Ensure 'totals' contains the same indices as rda keys
df_compare = pd.DataFrame({
    "Intake": totals,
    "RDA": pd.Series(rda)
}).dropna()

df_compare["% of RDA"] = (df_compare["Intake"] / df_compare["RDA"] * 100).round(1)

# 3️⃣ Print comparison
print("Comparison of Daily Intake to Recommendations:")
print(df_compare)

Comparison of Daily Intake to Recommendations:
                  Intake     RDA  % of RDA
Calcium (mg)       440.3  1000.0      44.0
Carbohydrate (g)   132.5   130.0     101.9
Copper (mg)          2.3     0.9     255.6
Energy (kcal)     1648.7  2500.0      65.9
Fibre (g)           31.0    38.0      81.6
Iron (mg)           14.3     8.0     178.8
Magnesium (mg)     485.0   420.0     115.5
Phosphorus (mg)   2111.1   700.0     301.6
Potassium (mg)    5145.7  3400.0     151.3
Protein (g)        179.2    56.0     320.0
Sodium (mg)       1852.9  2300.0      80.6
Zinc (mg)           10.2    11.0      92.7
